# Building Synthetic Agents!!! March 7 20201

In [1]:
#BuildingInSimulationAndStats

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import pyproj

pyproj.datadir.set_data_dir('/Users/shai/anaconda3/envs/geo_env/share/proj')




In [2]:
BldInfo = pd.read_excel('BuildingInSimulationAndStats.xlsx')

In [3]:
BldInfo = BldInfo.iloc[:,1:]

In [93]:
pd.options.display.max_rows=150
pd.options.display.max_columns=150

In [114]:
#Low_discount,High_discount

In [156]:
BldInfo['Owner_Percent'].mean()

0.647737967684634

In [86]:
BldInfo

,BeforeBldgs,AfterBldgs,ProjType,ProjNumber,OrderA,OrderB,OrderC,plan_num,Row Labels,min_living_till_2020,max_living_till_2020,avg_living_till_2020,Min_age_2020,Max_age_2020,Average_age_2020,StdDev_age_2020,Before_app,Above_65,Under_65,percent_above_65,Area_round_max,Area_round_mean,Area_round_min,Area_round_mode,Area_round_std,Discount_sum,High_discount_sum,Low_Discount_35_sum,No_Discount_sum,Owner_sum,Renter_sum,discount_rate_mean,total_appartments,discount_percent,Low_discount,High_discount,No_discount,Owner_Percent,houseHold2
0,210_19,210_19_a,1,20180988,1,1,17,20180988,1.0,2.000000,35.000000,9.000000,5.000000,82.000000,45.000000,19.000000,20.000000,8.000000,47.000000,14.545455,80.000000,63.000000,30.000000,70.000000,16.000000,5.000000,4.000000,1.000000,15.000000,12.000000,8.000000,0.150829,20.000000,0.250000,0.050000,0.200000,0.750000,0.600000,2.750000
1,210_14,210_14_a,1,20100133,2,2,18,20100133,1.0,2.000000,57.000000,19.000000,2.000000,98.000000,48.000000,24.000000,16.000000,16.000000,46.000000,25.806452,90.000000,77.000000,70.000000,70.000000,8.000000,12.000000,8.000000,4.000000,4.000000,10.000000,6.000000,0.480824,16.000000,0.750000,0.250000,0.500000,0.250000,0.625000,3.875000
2,208_32,208_32_a,1,20180812,3,3,19,20180812,1.0,2.000000,33.000000,10.000000,5.000000,89.000000,46.000000,23.000000,17.000000,11.000000,24.000000,31.428571,90.000000,62.000000,30.000000,50.000000,17.000000,7.000000,4.000000,3.000000,10.000000,10.000000,7.000000,0.284133,17.000000,0.411765,0.176471,0.235294,0.588235,0.588235,2.058824
3,210_3,210_3_a,1,20181092,4,4,20,20181092,1.0,2.000000,54.000000,15.000000,2.000000,99.000000,45.000000,26.000000,16.000000,17.000000,42.000000,28.813559,80.000000,80.000000,80.000000,80.000000,0.000000,7.000000,6.000000,1.000000,9.000000,10.000000,6.000000,0.285945,16.000000,0.437500,0.062500,0.375000,0.562500,0.625000,3.687500
4,210_35,210_35_r,2,20171974,5,20,13,20171974,1.0,2.000000,43.000000,10.000000,3.000000,90.000000,46.000000,23.000000,13.000000,14.000000,39.000000,26.415094,100.000000,66.000000,30.000000,60.000000,16.000000,5.000000,3.000000,2.000000,8.000000,8.000000,5.000000,0.243069,13.000000,0.384615,0.153846,0.230769,0.615385,0.615385,4.076923
5,210_12,210_12_a,1,20180345,6,5,21,20180345,1.0,2.000000,48.000000,17.000000,3.000000,104.000000,55.000000,20.000000,39.000000,38.000000,60.000000,38.775510,90.000000,69.000000,40.000000,80.000000,15.000000,18.000000,10.000000,8.000000,21.000000,27.000000,12.000000,0.226707,39.000000,0.461538,0.205128,0.256410,0.538462,0.692308,2.512821
6,210_10,210_10_a,1,20140556,7,6,22,20140556,1.0,2.000000,54.000000,15.000000,2.000000,109.000000,50.000000,24.000000,61.000000,55.000000,99.000000,35.714286,80.000000,62.000000,40.000000,70.000000,9.000000,30.000000,18.000000,12.000000,31.000000,39.000000,22.000000,0.296857,61.000000,0.491803,0.196721,0.295082,0.508197,0.639344,2.524590
7,"211_19,211_20,211_19","211_22_r,211_20_r,211_19_r",2,20170076,8,21,14,20170076,1.0,2.000000,12.000000,6.000000,2.000000,72.000000,27.000000,26.000000,4.000000,2.000000,6.000000,25.000000,70.000000,57.000000,50.000000,50.000000,9.000000,2.000000,2.000000,0.000000,2.000000,2.000000,2.000000,0.500000,4.000000,0.500000,0.000000,0.500000,0.500000,0.500000,2.000000
8,"224_3,220_7,220_3,210_34,210_36,220_5,210_32_2...","220_3_224_3_rr,210_30_210_32_rr,220_5_rr,220_7...",3,502-0196659,9,24,1,502-0196659,8.0,2.125000,48.875000,15.875000,4.375000,98.875000,47.875000,23.625000,20.250000,18.375000,43.375000,29.204431,128.750000,77.000000,53.750000,73.750000,20.625000,8.750000,5.750000,3.000000,11.375000,10.375000,9.750000,0.258501,20.125000,0.435034,0.160623,0.274411,0.564966,0.544690,3.097580
9,215_6,215_6_r,2,20150857,10,22,15,20150857,1.0,2.000000,58.000000,14.000000,2.000000,104.000000,45.000000,26.000000,12.000000,12.000000,30.000000,28.571429,100.000000,88.000000,60.000000,90.000000,14.000000,6.000000,3.000000,3.000000,6.000000,8.000000,4.000000,0.254921,12.000000,0.500000,0.250000,0.250000,0.5000

16    208_12
28     220_4
Name: BeforeBldgs, dtype: object

In [27]:
BldInfo['Before_app'].sum()

647.8820518063217

In [249]:
currentBLdg

BeforeBldgs                  201_75
AfterBldgs                201_75_rr
ProjType                          3
ProjNumber              502-0368860
OrderA                           26
OrderB                           26
OrderC                            3
plan_num                        NaN
Row Labels                      NaN
min_living_till_2020       2.829422
max_living_till_2020      47.140189
avg_living_till_2020       10.70332
Min_age_2020               3.746441
Max_age_2020              99.726437
Average_age_2020          47.275808
StdDev_age_2020           23.340628
Before_app                15.993292
Above_65                   8.718933
Under_65                  24.715018
percent_above_65           27.81706
Area_round_max           149.883794
Area_round_mean           82.769758
Area_round_min            53.735872
Area_round_mode            86.02142
Area_round_std            23.848454
Discount_sum              12.160586
High_discount_sum           2.83419
Low_Discount_35_sum        0

In [250]:
currentBLdg['discount_percent'].mean()

0.537829221701557

In [270]:
def minMaxColumnsCreator(tuple_,DS):
    DS[tuple_[0]] = 1
    option = tuple_[-1]
    if (option == 'between'):
        min_ = currentBLdg[tuple_[1]].astype(int)
        max_ = currentBLdg[tuple_[2]].astype(int)
        DS[tuple_[0]] = DS[tuple_[0]].apply(lambda x: np.random.randint(min_,max_))
    else: #insert
        DS[tuple_[0]] = currentBLdg['Area_round_mode'].astype(int)
        
    return DS[tuple_[0]]

In [271]:
def percentColumnImplementation(tuple_,DS):
    DS[tuple_[0]] = 1
    option = tuple_[-1]
    posNumber = (currentBLdg[tuple_[1]]*currentBLdg['Before_app']).astype(int)
    posList  = DS['apIndex'].sample(n=posNumber).tolist()
    negList = DS[~DS['apIndex'].isin(posList)]['apIndex'].tolist()
    DS.loc[DS['apIndex'].isin(posList),tuple_[0]]=1
    DS.loc[DS['apIndex'].isin(negList),tuple_[0]]=0
    if (option=='discount'): #discount
        new100 = currentBLdg['Low_discount']+currentBLdg['High_discount']
        if new100>0:
            highP = currentBLdg['High_discount']/new100
            discountAppsCon = DS['apIndex'].isin(posList)
            highDiscountList = DS[discountAppsCon]['apIndex'].sample(frac=highP).tolist()
            lowDiscountList = DS[discountAppsCon][~DS[discountAppsCon]['apIndex'].isin(highDiscountList)]['apIndex'].tolist()
            DS.loc[DS['apIndex'].isin(highDiscountList),'DiscountType']='High'
            DS.loc[DS['apIndex'].isin(lowDiscountList),'DiscountType']='Low'
            DS.loc[DS['apIndex'].isin(negList),'DiscountType'] = 'Non'
    return DS

In [272]:
def syntehticAgentsFromBuilding(bld,numAp,currentBLdg):
    DS = pd.DataFrame({'Building ID':[bld]*numAp,'apIndex':range(1,numAp+1)})
    listData =[('YearsInBldg','min_living_till_2020','max_living_till_2020','between'),('avrgAge','Min_age_2020','Max_age_2020','between'),('AptArea','Area_round_mode','insert')]
    for tuple_ in listData:
        DS[tuple_[0]] = minMaxColumnsCreator(tuple_,DS.copy())
        
    listData2 = [('Owner','Owner_Percent','simple'),('Discount','discount_percent','DiscountType','Low_discount','High_discount','discount')]
    for tuple_ in listData2:
        DS = percentColumnImplementation(tuple_,DS.copy())

    return DS
    

In [273]:
for i in BldInfo.index:
    currentBLdg = BldInfo.loc[i]
    bldProj = currentBLdg['ProjNumber']
    numAp = currentBLdg['Before_app'].astype(int)
    listOfBuildings = currentBLdg['BeforeBldgs'].split(',')
    lenBuildings = len(listOfBuildings)
    numAp = currentBLdg['Before_app'].astype(int)
    totalAp = lenBuildings*numAp
    if i==0: #first run
        for bld in listOfBuildings:
            if bld == listOfBuildings[0]:
                     
                apDs = syntehticAgentsFromBuilding(bld,numAp,currentBLdg)
            else: 
                apDs = pd.concat([apDs,syntehticAgentsFromBuilding(bld,numAp,currentBLdg)])
    else:
        for bld in listOfBuildings:
            if bld == listOfBuildings[0]:
                currentBldg = syntehticAgentsFromBuilding(bld,numAp,currentBLdg)
            else:
                currentBldg = pd.concat([currentBldg,syntehticAgentsFromBuilding(bld,numAp,currentBLdg)])
        apDs = pd.concat([apDs,currentBldg]).reset_index(drop=True)
apDs['BldDoor'] =apDs['Building ID']+"-"+apDs['apIndex'].astype(str)
    

In [275]:
apDs.head(25)

,Building ID,apIndex,YearsInBldg,avrgAge,AptArea,Owner,Discount,DiscountType,BldDoor
0,210_19,1,28,29,70,1,0,Non,210_19-1
1,210_19,2,30,55,70,0,0,Non,210_19-2
2,210_19,3,28,42,70,0,0,Non,210_19-3
3,210_19,4,14,54,70,1,1,High,210_19-4
4,210_19,5,15,75,70,1,0,Non,210_19-5
5,210_19,6,19,6,70,0,1,High,210_19-6
6,210_19,7,5,17,70,1,0,Non,210_19-7
7,210_19,8,24,76,70,0,1,High,210_19-8
8,210_19,9,15,53,70,0,0,Non,210_19-9
9,210_19,10,32,61,70,1,0,Non,210_19-10
